<a href="https://colab.research.google.com/github/Beatriz-Yun/AIFFEL_LMS/blob/main/Exploration/%5BE-09%5DMovielens%EC%98%81%ED%99%94%EC%B6%94%EC%B2%9C%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Movielens 영화 추천 실습
- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
- 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해볼 수 있습니다.
- 별점을 시청횟수로 해석해서 생각하겠습니다.
- 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

## 1) 데이터 준비와 전처리
Movielens 데이터는 **rating.dat** 안에 이미 인덱싱까지 완료된 **사용자-영화-평점 데이터**가 깔끔하게 정리되어 있습니다.

In [ ]:
import pandas as pd
import os

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)    # 원본데이터 길이

ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [ ]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)    # 전처리한 데이터 길이

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [ ]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [ ]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [ ]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')

movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


## 2) 데이터 분석
- ratings에 있는 유니크한 영화 개수
- rating에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)

In [ ]:
# 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [ ]:
# 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [ ]:
# 가장 인기 있는 영화 30개
movies_count = ratings.groupby('movie_id')['user_id'].count()

movies_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

## 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.

In [ ]:
# 내가 선호하는 영화 5개
my_favorite = ['Little Mermaid, The (1989)' , 'Cinderella (1950)' ,'Sound of Music, The (1965)' ,'Indiana Jones and the Temple of Doom (1984)' ,'Toy Story (1995)']

for i in range(5):
    temp = movies[movies['title'] == my_favorite[i]]
    print(temp)

      movie_id                       title  \
2012      2081  Little Mermaid, The (1989)   

                                            genre  
2012  Animation|Children's|Comedy|Musical|Romance  
      movie_id              title                         genre
1009      1022  Cinderella (1950)  Animation|Children's|Musical
      movie_id                       title    genre
1022      1035  Sound of Music, The (1965)  Musical
      movie_id                                        title             genre
2046      2115  Indiana Jones and the Temple of Doom (1984)  Action|Adventure
   movie_id             title                        genre
0         1  Toy Story (1995)  Animation|Children's|Comedy


In [ ]:
my_favorite = [2081, 1022, 1035, 2115, 1]
my_count = [5.0, 4.0, 5.0, 3.0, 4.0]
my_movielist = pd.DataFrame({'user_id': ['sehui']*5, 'movie_id': my_favorite, 'count': my_count})

if not ratings.isin({'user_id':['sehui']})['user_id'].any():  # user_id에 'sehui'이라는 데이터가 없다면
    ratings = ratings.append(my_movielist)                           # 위에 임의로 만든 my_movielist 데이터를 추가해 줍니다. 

ratings.tail(10)

,user_id,movie_id,count,timestamp
1000203,6040,1090,3.0,956715518.0
1000205,6040,1094,5.0,956704887.0
1000206,6040,562,5.0,956704746.0
1000207,6040,1096,4.0,956715648.0
1000208,6040,1097,4.0,956715569.0
0,sehui,2081,5.0,NaN
1,sehui,1022,4.0,NaN
2,sehui,1035,5.0,NaN
3,sehui,2115,3.0,NaN
4,sehui,1,4.0,NaN


### 추가적인 전처리 (indexing, 칼럼제거)

In [ ]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = data['user_id'].unique()
artist_unique = data['artist'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
artist_to_idx = {v:k for k,v in enumerate(artist_unique)}

## 4) CSR matrix를 직접 만들어 봅시다.

## 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

## 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

## 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

## 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.